In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# Load data
df = pd.read_csv(r'C:\Users\kau75421\ITSM SLA Optimzers\data\raw\itsm_sla_augmented_for_recommender.csv')
df

,Ticket ID,Created At,Created Date,Responded At,Resolved At,Priority,Status,SLA Breach,Escalation Level,Ticket Type,...,Ticket Source,Impact Level,Change Request Linked,Problem Ticket Linked,Technical Domain,Primary Tool Used,Certifications,Years of Experience,Tickets Resolved Past Month,Resolution Success Rate
0,TKT000001,2025-03-18 09:55:25,2025-03-18,2025-03-18 11:27:25,2025-03-20 21:27:25,P1,Reopened,False,Level 2,Request,...,Automated Monitoring,Critical,Yes,No,Cloud,Ansible,RHCE,8,8,0.93
1,TKT000002,2025-03-18 02:47:44,2025-03-18,2025-03-18 05:42:44,2025-03-20 18:42:44,P4,Reopened,False,Level 3,Request,...,Email,Low,No,Yes,Network,Ansible,AWS,15,8,0.78
2,TKT000003,2025-03-05 18:24:24,2025-03-05,2025-03-05 20:21:24,2025-03-06 10:21:24,P3,Resolved,False,NaN,Change,...,Email,Medium,No,No,Network,Nagios,Cisco,13,17,0.90
3,TKT000004,2025-04-04 03:23:27,2025-04-04,2025-04-04 07:22:27,2025-04-06 05:22:27,P3,Closed,False,Level 1,Incident,...,Portal,High,Yes,Yes,Database,Zabbix,ITIL,5,5,0.79
4,TKT000005,2025-06-18 18:56:46,2025-06-18,2025-06-18 23:50:46,2025-06-20 18:50:46,P1,Reopened,False,Level 1,Request,...,Portal,Low,No,Yes,Hardware,Zabbix,Cisco,8,4,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,TKT049996,2025-03-13 06:16:32,2025-03-13,2025-03-13 11:11:32,2025-03-15 23:11:32,P2,Reopened,True,Level 1,Problem,...,Phone,Critical,No,Yes,Security,Kibana,CompTIA,1,4,0.80
49996,TKT049997,2025-06-11 05:13:52,2025-06-11,2025-06-11 06:25:52,2025-06-11 09:25:52,P3,In Progress,True,Level 3,Problem,...,Portal,Medium,No,Yes,Database,Kibana,RHCE,9,11,0.80
49997,TKT049998,2024-12-29 00:01:10,2024-12-29,2024-12-29 04:21:10,2024-12-30 10:21:10,P2,Open,False,Level 1,Problem,...,Email,Low,No,Yes,Network,ServiceNow,CompTIA,5,13,0.89
49998,TKT049999,2025-04-11 22:07:38,2025-04-11,2025-04-11 22:27:38,2025-04-14 14:27:38,P4,Reopened,True,Level 1,Incident,...,Automated Monitoring,Low,Yes,No,Database,Wireshark,Cisco,12,8,0.83


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
import pandas as pd

def clean_itsm_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans and imputes ITSM SLA ticket dataset.

    Args:
        df (pd.DataFrame): Raw ITSM SLA dataset

    Returns:
        pd.DataFrame: Cleaned and imputed DataFrame
    """
    df = df.copy()

    # Parse datetime fields
    datetime_cols = ["Created At", "Responded At", "Resolved At"]
    for col in datetime_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Fill missing categorical values with "Unknown"
    cat_fill = [
        "Technician Skill Level", "Customer Segment", "Priority",
        "Ticket Type", "Region", "Impact Level",
        "Technical Domain", "Primary Tool Used", "Certifications"
    ]
    for col in cat_fill:
        if col in df.columns:
            df[col] = df[col].fillna("Unknown")

    # Fill missing numeric values with median or safe defaults
    num_fill_median = [
        "Resolution Time (hours)", "MTTR (hours)", "MTBF (days)",
        "Customer Satisfaction Score (CSAT)"
    ]
    for col in num_fill_median:
        if col in df.columns:
            df[col] = df[col].fillna(df[col].median())

    # Additional derived time-based features
    df["Response Delay (minutes)"] = (df["Responded At"] - df["Created At"]).dt.total_seconds() / 60.0
    df["Resolution Duration (hours)"] = (df["Resolved At"] - df["Created At"]).dt.total_seconds() / 3600.0

    df["Response Delay (minutes)"] = df["Response Delay (minutes)"].fillna(df["Response Delay (minutes)"].median())
    df["Resolution Duration (hours)"] = df["Resolution Duration (hours)"].fillna(df["Resolution Duration (hours)"].median())

    return df

df_cleaned = clean_itsm_data(df)
df_cleaned

,Ticket ID,Created At,Created Date,Responded At,Resolved At,Priority,Status,SLA Breach,Escalation Level,Ticket Type,...,Change Request Linked,Problem Ticket Linked,Technical Domain,Primary Tool Used,Certifications,Years of Experience,Tickets Resolved Past Month,Resolution Success Rate,Response Delay (minutes),Resolution Duration (hours)
0,TKT000001,2025-03-18 09:55:25,2025-03-18,2025-03-18 11:27:25,2025-03-20 21:27:25,P1,Reopened,False,Level 2,Request,...,Yes,No,Cloud,Ansible,RHCE,8,8,0.93,92.0,59.533333
1,TKT000002,2025-03-18 02:47:44,2025-03-18,2025-03-18 05:42:44,2025-03-20 18:42:44,P4,Reopened,False,Level 3,Request,...,No,Yes,Network,Ansible,AWS,15,8,0.78,175.0,63.916667
2,TKT000003,2025-03-05 18:24:24,2025-03-05,2025-03-05 20:21:24,2025-03-06 10:21:24,P3,Resolved,False,NaN,Change,...,No,No,Network,Nagios,Cisco,13,17,0.90,117.0,15.950000
3,TKT000004,2025-04-04 03:23:27,2025-04-04,2025-04-04 07:22:27,2025-04-06 05:22:27,P3,Closed,False,Level 1,Incident,...,Yes,Yes,Database,Zabbix,ITIL,5,5,0.79,239.0,49.983333
4,TKT000005,2025-06-18 18:56:46,2025-06-18,2025-06-18 23:50:46,2025-06-20 18:50:46,P1,Reopened,False,Level 1,Request,...,No,Yes,Hardware,Zabbix,Cisco,8,4,0.72,294.0,47.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,TKT049996,2025-03-13 06:16:32,2025-03-13,2025-03-13 11:11:32,2025-03-15 23:11:32,P2,Reopened,True,Level 1,Problem,...,No,Yes,Security,Kibana,CompTIA,1,4,0.80,295.0,64.916667
49996,TKT049997,2025-06-11 05:13:52,2025-06-11,2025-06-11 06:25:52,2025-06-11 09:25:52,P3,In Progress,True,Level 3,Problem,...,No,Yes,Database,Kibana,RHCE,9,11,0.80,72.0,4.200000
49997,TKT049998,2024-12-29 00:01:10,2024-12-29,2024-12-29 04:21:10,2024-12-30 10:21:10,P2,Open,False,Level 1,Problem,...,No,Yes,Network,ServiceNow,CompTIA,5,13,0.89,260.0,34.333333
49998,TKT049999,2025-04-11 22:07:38,2025-04-11,2025-04-11 22:27:38,2025-04-14 14:27:38,P4,Reopened,True,Level 1,Incident,...,Yes,No,Database,Wireshark,Cisco,12,8,0.83,20.0,64.333333


In [ ]:
import numpy as np

def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generate technician-level and ticket-level features for ranking.

    Args:
        df (pd.DataFrame): Cleaned ITSM dataset

    Returns:
        pd.DataFrame: Feature-engineered dataset
    """
    df = df.copy()

    # Generate SLA breach label
    df["sla_breach_label"] = df["SLA Breach"].astype(int)
    df["label"] = 1 - df["sla_breach_label"]  # 1 = successful

    # Technician historical performance
    tech_stats = (
        df.groupby("Assigned Technician ID")
        .agg(
            tech_total_tickets=("Ticket ID", "count"),
            tech_breach_rate=("sla_breach_label", "mean"),
            tech_avg_resolution_time=("Resolution Time (hours)", "mean"),
            tech_avg_csat=("Customer Satisfaction Score (CSAT)", "mean"),
            tech_experience=("Years of Experience", "max"),
            tech_monthly_volume=("Tickets Resolved Past Month", "max"),
            tech_resolution_success=("Resolution Success Rate", "max")
        )
        .reset_index()
    )
    tech_stats["tech_success_rate"] = 1 - tech_stats["tech_breach_rate"]

    # Merge into original dataframe
    df = df.merge(tech_stats, on="Assigned Technician ID", how="left")

    # Derived flags or ratios (optional)
    df["reopen_ratio"] = df["Reopened Count"] / (df["tech_total_tickets"] + 1)
    df["efficiency"] = df["tech_avg_resolution_time"] / (df["Resolution Duration (hours)"] + 1e-5)

    return df

df_engineered = engineer_features(df_cleaned)
df_engineered.columns


Index(['Ticket ID', 'Created At', 'Created Date', 'Responded At',
       'Resolved At', 'Priority', 'Status', 'SLA Breach', 'Escalation Level',
       'Ticket Type', 'Root Cause Category', 'Reopened Count', 'Service Name',
       'Service Category', 'Service SLA Target', 'Customer ID',
       'Customer Segment', 'Contract Type', 'Region', 'Business Unit',
       'Assigned Technician ID', 'Technician Skill Level', 'Team ID',
       'Escalation Team ID', 'Response Time (minutes)',
       'Resolution Time (hours)', 'MTTR (hours)', 'MTBF (days)',
       'SLA Compliance Flag', 'Penalty Cost',
       'Customer Satisfaction Score (CSAT)', 'Ticket Source', 'Impact Level',
       'Change Request Linked', 'Problem Ticket Linked', 'Technical Domain',
       'Primary Tool Used', 'Certifications', 'Years of Experience',
       'Tickets Resolved Past Month', 'Resolution Success Rate',
       'Response Delay (minutes)', 'Resolution Duration (hours)',
       'sla_breach_label', 'label', 'tech_total_ti

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def encode_categorical_features(df: pd.DataFrame, categorical_cols: list):
    """
    Encode categorical columns using LabelEncoder.

    Args:
        df (pd.DataFrame): DataFrame with raw categorical columns
        categorical_cols (list): List of column names to encode

    Returns:
        pd.DataFrame: Transformed DataFrame
        dict: Dictionary of fitted LabelEncoders keyed by column name
    """
    df = df.copy()
    encoders = {}

    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        encoders[col] = le

    return df, encoders

# Define categorical columns to encodeencode


categorical_columns = [
    "Priority", "Ticket Type", "Region", "Impact Level",
    "Technician Skill Level", "Customer Segment",
    "Technical Domain", "Primary Tool Used", "Certifications"
]

df_encoded, encoders = encode_categorical_features(df_engineered, categorical_columns)

In [ ]:
df_encoded.columns

Index(['Ticket ID', 'Created At', 'Created Date', 'Responded At',
       'Resolved At', 'Priority', 'Status', 'SLA Breach', 'Escalation Level',
       'Ticket Type', 'Root Cause Category', 'Reopened Count', 'Service Name',
       'Service Category', 'Service SLA Target', 'Customer ID',
       'Customer Segment', 'Contract Type', 'Region', 'Business Unit',
       'Assigned Technician ID', 'Technician Skill Level', 'Team ID',
       'Escalation Team ID', 'Response Time (minutes)',
       'Resolution Time (hours)', 'MTTR (hours)', 'MTBF (days)',
       'SLA Compliance Flag', 'Penalty Cost',
       'Customer Satisfaction Score (CSAT)', 'Ticket Source', 'Impact Level',
       'Change Request Linked', 'Problem Ticket Linked', 'Technical Domain',
       'Primary Tool Used', 'Certifications', 'Years of Experience',
       'Tickets Resolved Past Month', 'Resolution Success Rate',
       'Response Delay (minutes)', 'Resolution Duration (hours)',
       'sla_breach_label', 'label', 'tech_total_ti

In [80]:
selected_features = [
    "Priority", "Ticket Type", "Region", "Impact Level",
    "Technician Skill Level", "Customer Segment",
    "Reopened Count", "Response Time (minutes)", "Resolution Time (hours)",
    "MTTR (hours)", "MTBF (days)", "tech_success_rate",
    "tech_avg_resolution_time", "tech_avg_csat",
    "Years of Experience", "Tickets Resolved Past Month", "Resolution Success Rate"
]


In [82]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

def train_lambdarank_model(
    df: pd.DataFrame,
    features: list,
    query_col: str = "Ticket ID",
    doc_col: str = "Assigned Technician ID",
    label_col: str = "label",
    model_path: str = "lambdarank_sla_model.txt"
):
    # Step 1: Add query + doc columns
    df = df.copy()
    df["query_id"] = df[query_col]
    df["doc_id"] = df[doc_col]

    # Step 2: Split on unique tickets
    train_queries, test_queries = train_test_split(
        df["query_id"].unique(), test_size=0.2, random_state=42
    )

    train_df = df[df["query_id"].isin(train_queries)].reset_index(drop=True)
    test_df = df[df["query_id"].isin(test_queries)].reset_index(drop=True)

    # Step 3: Group sizes for LightGBM
    train_group = train_df.groupby("query_id").size().to_list()
    test_group = test_df.groupby("query_id").size().to_list()

    lgb_train = lgb.Dataset(train_df[features], label=train_df[label_col], group=train_group)
    lgb_test = lgb.Dataset(test_df[features], label=test_df[label_col], group=test_group)

    # Step 4: Training params
    params = {
        "objective": "lambdarank",
        "metric": "ndcg",
        "ndcg_eval_at": [1, 3, 5],
        "learning_rate": 0.05,
        "num_leaves": 31,
        "min_data_in_leaf": 20,
        "verbose": -1,
        "random_state": 42
    }

    model = lgb.train(
        params,
        lgb_train,
        valid_sets=[lgb_train, lgb_test],
        valid_names=["train", "valid"],
    )

    # Save model
    model.save_model(model_path)
    print(f" Model saved to: {model_path}")

    return model, train_df, test_df


In [81]:
import pandas as pd

# Example dummy DataFrame structure


# Step 1: Define features used in model


# Step 2: Call the function
model, train_df, test_df = train_lambdarank_model(
    df=df_encoded,
    features=selected_features,
    query_col="Ticket ID",
    doc_col="Assigned Technician ID",
    label_col="label",
    model_path="lambdarank_sla_model.txt"
)

✅ Model saved to: lambdarank_sla_model.txt


In [78]:
import pandas as pd
import lightgbm as lgb

df = df_encoded
model = lgb.Booster(model_file="lambdarank_sla_model.txt")  # trained model

# --- Step 2: Ensure features and label exist ---
selected_features = [
    "Priority", "Ticket Type", "Region", "Impact Level",
    "Technician Skill Level", "Customer Segment",
    "Reopened Count", "Response Time (minutes)", "Resolution Time (hours)",
    "MTTR (hours)", "MTBF (days)", "tech_success_rate",
    "tech_avg_resolution_time", "tech_avg_csat",
    "Years of Experience", "Tickets Resolved Past Month", "Resolution Success Rate"
]

# Add label column if missing
if "label" not in df.columns:
    df["label"] = (~df["SLA Breach"]).astype(int)

# --- Step 3: Predict for all ticket-tech pairs ---
X_all = df[selected_features]
df["predicted_score"] = model.predict(X_all)

# --- Step 4: Get top 3 technicians for a specific Ticket ID ---
ticket_id = "TKT000005"
ticket_result = (
    df[df["Ticket ID"] == ticket_id]
    .sort_values("predicted_score", ascending=False)
    .head(3)
)

# --- Step 5: Show result ---
display_columns = ["Assigned Technician ID", "predicted_score"] + selected_features
print(ticket_result[display_columns])


  Assigned Technician ID  predicted_score  Priority  Ticket Type  Region  \
4                TECH205              0.0         0            3       0   

   Impact Level  Technician Skill Level  Customer Segment  Reopened Count  \
4             2                       1                 2               1   

   Response Time (minutes)  Resolution Time (hours)  MTTR (hours)  \
4                      294                       43         33.65   

   MTBF (days)  tech_success_rate  tech_avg_resolution_time  tech_avg_csat  \
4        79.08           0.916667                 39.541667       2.832917   

   Years of Experience  Tickets Resolved Past Month  Resolution Success Rate  
4                    8                            4                     0.72  


In [79]:
ticket_id = "TKT000005"
ticket_result = (
    test_df[test_df["Ticket ID"] == ticket_id]
    .sort_values("predicted_score", ascending=False)
)

print(ticket_result[["Assigned Technician ID", "predicted_score"] + selected_features])


  Assigned Technician ID  predicted_score  Priority  Ticket Type  Region  \
0                TECH205              0.0         0            3       0   

   Impact Level  Technician Skill Level  Customer Segment  Reopened Count  \
0             2                       1                 2               1   

   Response Time (minutes)  Resolution Time (hours)  MTTR (hours)  \
0                      294                       43         33.65   

   MTBF (days)  tech_success_rate  tech_avg_resolution_time  tech_avg_csat  \
0        79.08           0.916667                 39.541667       2.832917   

   Years of Experience  Tickets Resolved Past Month  Resolution Success Rate  
0                    8                            4                     0.72  
